# Exam 1

This activity is intended to assess your knowledge
in formulating engineering problems as univariate root-finding tasks,
as well as in applying relevant numerical methods to solve them.

## Scenario

Two batching plants -- Bonnie and Clyde --
have respective ratings of
$p_{1}$ kilowatts and $q_{2}$ kilovars,
have respective lagging power factors of
$\phi_{1}$ and $\phi_{2}$,
and
are separately served by dedicated feeders
whose respective impedances are
$r_{1} + j x_{1}$
and
$r_{2} + j x_{2}$ ohms.
The feeders branch out from a substation bus
regulated around a nominal voltage of $u$ kilovolts.

The main objective is to determine the phase-angle difference
between the rated operating voltages of Bonnie and of Clyde.
One way to accomplish the main objective is
to solve for the operating voltage magnitudes of Bonnie and of Clyde
(say, $v_{1}$ and $v_{2}$, respectively).
With such values,
determining the desired phase-angle difference
is a matter of analysis and complex-number arithmetic.

## Part I

This is the Written component of the exam.

### Question 1

Derive a univariate function $f\!\left(v\right)$ whose root is $v_{1}$.

### Question 2

Derive a univariate function $g\!\left(v\right)$ whose root is $v_{2}$.

### Question 3

Describe how to calculate the desired phase-angle difference
using $v_{1}$ and $v_{2}$.

### Question 4

If you are to solve for $v_{1}$ using the bisection method,
how will you set the search interval?

### Question 5

If you are to solve for $v_{2}$ using the Newton-Raphson method,
how will you set the initial estimate?

### Question 6

If you are to solve for $v_{1}$ using the Newton-Raphson method,
derive an iteration scheme of the form
$v_{k+1} \gets v_{k} + h\!\left(v_{k}\right)$.

### Question 7

Reason out whether or not the desired phase-angle difference can take on multiple values.

### Question 8

In your stay so far in the university,
what do you consider to be the most profound
thing you have heard or learned from a teacher?

## Part II

This is the Programming component of the exam.

Set the power ratings as follows.
- M34W12 class:
  $p_{1} = 450$, $\phi_{1} = 0.87$, $q_{2} = 300$, and $\phi_{2} = 0.90$
- M34W34 class:
  $p_{1} = 375$, $\phi_{1} = 0.95$, $q_{2} = 325$, and $\phi_{2} = 0.80$
- H34W56 class:
  $p_{1} = 425$, $\phi_{1} = 0.75$, $q_{2} = 275$, and $\phi_{2} = 0.75$
- H34W78 class:
  $p_{1} = 301$, $\phi_{1} = 0.73$, $q_{2} = 334$, and $\phi_{2} = 0.84$

Set the feeder line parameters as follows.
- M34W12 class:
  $r_{1} = 0.97$, $x_{1} = 2.45$, $r_{2} = 0.99$, and $x_{2} = 2.51$
- M34W34 class:
  $r_{1} = 0.95$, $x_{1} = 2.54$, $r_{2} = 1.01$, and $x_{2} = 2.75$
- H34W56 class:
  $r_{1} = 1.01$, $x_{1} = 2.75$, $r_{2} = 0.95$, and $x_{2} = 2.54$
- H34W78 class:
  $r_{1} = 0.99$, $x_{1} = 2.51$, $r_{2} = 0.97$, and $x_{2} = 2.45$

Adopting real-world conditions,
the substation bus voltage is allowed to vary within 10 % of the nominal.
(Exceeding this range indicates undervoltage or overvoltage.)
Set the substation bus voltage as follows.
- M34W12 class: the minimum allowed for a $u = 49.0$
- M34W34 class: the maximum allowed for a $u = 13.8$
- H34W56 class: the minimum allowed for a $u = 13.8$
- H34W78 class: the maximum allowed for a $u = 49.0$

### Preliminaries

In [1]:
import math as mt

import scipy.optimize as spo

Define the following Python functions.
- `f()`, implementing $f\!\left(v\right)$
- `g()`, implementing $g\!\left(v\right)$
- `df()`, implementing the derivative of $f\!\left(v\right)$ w.r.t. $v$
- `dg()`, implementing the derivative of $g\!\left(v\right)$ w.r.t. $v$

Each of these functions should have the following inputs.
- `v`,
  a positional argument that represents the operating voltage
- `v_ss`,
  a keyword argument that represents the substation bus voltage
  and defaults to the nominal value

Lastly, define a Python function
`calc_angle_diff()` that returns the desired phase-angle difference
(in radians)
given two inputs:
- `v1`, the first positional argument representing $v_{1}$,
  and
- `v2`, the first positional argument representing $v_{2}$.

Replace the `pass` statements in the succeeding code cells.

In [2]:
def f(x):
    r1=0.99
    x1=2.51
    p1_kw=301.0
    pf1=0.73
    q2_kvar=334.0
    pf2=0.84
    U=(49.0/1.10)*1e3/mt.sqrt(3.0)

    P1=p1_kw*1e3
    Q1=P1*mt.tan(mt.acos(pf1))
    Q2=q2_kvar*1e3
    P2=Q2/mt.tan(mt.acos(pf2))

    P1ph=P1/3.0
    Q1ph=Q1/3.0
    P2ph=P2/3.0
    Q2ph=Q2/3.0

    a1=r1*(P1ph+P2ph)+x1*(Q1ph+Q2ph)
    b1=x1*(P1ph+P2ph)-r1*(Q1ph+Q2ph)

    v=float(x)
    return (v*v+a1)**2+b1*b1-(U*U)*(v*v)



In [3]:
def g(x,v1_mag):
    r2=0.97
    x2=2.45
    q2_kvar=334.0
    pf2=0.84

    Q2=q2_kvar*1e3
    P2=Q2/mt.tan(mt.acos(pf2))

    P2ph=P2/3.0
    Q2ph=Q2/3.0

    a2=r2*P2ph+x2*Q2ph
    b2=x2*P2ph-r2*Q2ph

    v=float(x)
    V1=float(v1_mag)
    return (v*v+a2)**2+b2*b2-(V1*V1)*(v*v)


In [4]:
def df(x):
    r1=0.99
    x1=2.51
    p1_kw=301.0
    pf1=0.73
    q2_kvar=334.0
    pf2=0.84
    U=(49.0/1.10)*1e3/mt.sqrt(3.0)

    P1=p1_kw*1e3
    Q1=P1*mt.tan(mt.acos(pf1))
    Q2=q2_kvar*1e3
    P2=Q2/mt.tan(mt.acos(pf2))

    P1ph=P1/3.0
    Q1ph=Q1/3.0
    P2ph=P2/3.0
    Q2ph=Q2/3.0

    a1=r1*(P1ph+P2ph)+x1*(Q1ph+Q2ph)

    v=float(x)
    return 2.0*v*(2.0*v*v+2.0*a1-U*U)


In [5]:
def dg(x,v1_mag):
    r2=0.97
    x2=2.45
    q2_kvar=334.0
    pf2=0.84

    Q2=q2_kvar*1e3
    P2=Q2/mt.tan(mt.acos(pf2))

    P2ph=P2/3.0
    Q2ph=Q2/3.0

    a2=r2*P2ph+x2*Q2ph

    v=float(x)
    V1=float(v1_mag)
    return 2.0*v*(2.0*v*v+2.0*a2-V1*V1)




In [6]:
def calc_angle_diff(v1,v2):
    r1=0.99
    x1=2.51
    r2=0.97
    x2=2.45
    p1_kw=301.0
    pf1=0.73
    q2_kvar=334.0
    pf2=0.84

    P1=p1_kw*1e3
    Q1=P1*mt.tan(mt.acos(pf1))
    Q2=q2_kvar*1e3
    P2=Q2/mt.tan(mt.acos(pf2))

    P1ph=P1/3.0
    Q1ph=Q1/3.0
    P2ph=P2/3.0
    Q2ph=Q2/3.0

    a1=r1*(P1ph+P2ph)+x1*(Q1ph+Q2ph)
    b1=x1*(P1ph+P2ph)-r1*(Q1ph+Q2ph)
    a2=r2*P2ph+x2*Q2ph
    b2=x2*P2ph-r2*Q2ph

    v1=float(max(v1,1e-9))
    v2=float(max(v2,1e-9))
    th1=-mt.atan2(b1,v1*v1+a1)
    th2=-mt.atan2(b2,v2*v2+a2)
    d=th1-th2
    if d<=-mt.pi or d>mt.pi:
        d=(d+mt.pi)%(2.0*mt.pi)-mt.pi
    return mt.degrees(d)



### Using the bisection method

In solving for $v_{1}$,
define Python variables `_V1L` and `_V1U`
to store the lower and the upper ends of the search interval.
Similarly define Python variables `_V2L` and `_V2U`
in solving for $v_{2}$.

Run 
[`scipy.optimize.bisect()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html)
such that you get
the (approximate) root
as well as information concerning the run.
Store the (approximate) root and the information in Python variables

- `v1_bs` and `v1_bs_info`, respectively, for $v_{1}$; and
- `v2_bs` and `v2_bs_info`, respectively, for $v_{2}$.

Set tolerances and iteration budget to their corresponding default values.

Replace the `pass` statement in the succeeding cells with the appropriate code.

In [7]:
U=(49.0/1.10)*1e3/mt.sqrt(3.0) 
v1L,v1U=0.5*U,U
v2L,v2U=0.5*U,U
v1_bs,v1_bs_info=spo.bisect(
    f,v1L,v1U,xtol=1e-6,maxiter=100,full_output=True,disp=False
)
v2_bs,v2_bs_info=spo.bisect(
    g,v2L,v2U,args=(v1_bs,),xtol=1e-6,maxiter=100,full_output=True,disp=False
)

In [8]:
print("-----")
print(v1_bs_info)
print("-----")
print(v2_bs_info)

-----
      converged: True
           flag: converged
 function_calls: 36
     iterations: 34
           root: 25687.75666667186
         method: bisect
-----
      converged: True
           flag: converged
 function_calls: 36
     iterations: 34
           root: 25670.615261604475
         method: bisect


In [9]:
print("Solving for Bonnie's operating voltage using the bisection method...")
print(f"\tinterval: {v1L/1e3:.3f} - {v1U/1e3:.3f} kilovolts")
print(f"\troot: {v1_bs/1e3:.6f}")
print(f"\tresidual: {f(v1_bs):.6e}")

Solving for Bonnie's operating voltage using the bisection method...
	interval: 12.859 - 25.718 kilovolts
	root: 25.687757
	residual: -1.626726e+07


In [10]:
print("Solving for Clyde's operating voltage using the bisection method...")
print(f"\tinterval: {v2L/1e3:.3f} - {v2U/1e3:.3f} kilovolts")
print(f"\troot: {v2_bs/1e3:.6f}")
print(f"\tresidual: {g(v2_bs, v1_bs):.6e}")

Solving for Clyde's operating voltage using the bisection method...
	interval: 12.859 - 25.718 kilovolts
	root: 25.670615
	residual: 8.138112e+06


In [11]:
print(f"Phase-angle difference: {calc_angle_diff(v1_bs, v2_bs):.6f} degrees")

Phase-angle difference: -0.014429 degrees


### Using the Newton-Raphson method

In solving for $v_{1}$,
define a Python variable `_V1NR`
to store the initial estimate.
Similarly define a Python variable `_V2NR`
in solving for $v_{2}$.

Run
[`scipy.optimize.newton()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html)
such that you get the (approximate) root as well as convergence information.
Store the (approximate) root and the information in Python variables

- `v1_nr` and `v1_nr_info`, respectively, for $v_{1}$; and
- `v2_nr` and `v2_nr_info`, respectively, for $v_{2}$.

Set tolerances and iteration budget to their corresponding default values.

Replace the `pass` statement in the succeeding cell with your code.

In [12]:
U=(49.0/1.10)*1e3/mt.sqrt(3.0)
_V1NR=0.90*U
_V2NR=0.90*U
tol,maxit=1e-8,100
v1_nr,v1_nr_info=spo.newton(
    f,x0=_V1NR,fprime=df,tol=tol,maxiter=maxit,full_output=True,disp=False
)
v2_nr,v2_nr_info=spo.newton(
    g,x0=_V2NR,fprime=dg,args=(v1_nr,),
    tol=tol,maxiter=maxit,full_output=True,disp=False
)

In [13]:
print("-----")
print(v1_nr_info)
print("-----")
print(v2_nr_info)

-----
      converged: True
           flag: converged
 function_calls: 12
     iterations: 6
           root: 25687.756667151712
         method: newton
-----
      converged: True
           flag: converged
 function_calls: 12
     iterations: 6
           root: 25670.61526184411
         method: newton


In [14]:
print(f"Solving for Bonnie's operating voltage using the Newton-Raphson method...")
print(f"\tstart: {_V1NR/1e3:.3f} kilovolts")
print(f"\troot: {v1_nr/1e3:.6f}")
print(f"\tresidual: {f(v1_nr):.6e}")

Solving for Bonnie's operating voltage using the Newton-Raphson method...
	start: 23.146 kilovolts
	root: 25.687757
	residual: 6.400000e+01


In [15]:
print(f"Solving for Clyde's operating voltage using the bisection method...")
print(f"\tstart: {_V2NR/1e3:.3f} kilovolts")
print(f"\troot: {v2_nr/1e3:.6f}")
print(f"\tresidual: {g(v2_nr, v1_nr):.6e}")

Solving for Clyde's operating voltage using the bisection method...
	start: 23.146 kilovolts
	root: 25.670615
	residual: -6.400000e+01


In [16]:
print(f"Phase-angle difference: {calc_angle_diff(v1_nr, v2_nr):.6f} degrees")

Phase-angle difference: -0.014429 degrees


### Using the secant method

In solving for $v_{1}$,
define a Python variable `_V1SC`
to store the initial estimate.
Similarly define a Python variable `_V2SC`
in solving for $v_{2}$.

Run
[`scipy.optimize.newton()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html)
such that you get the (approximate) root as well as information concerning the run.
Store the (approximate) root and the information in Python variables

- `v1_sc` and `v1_sc_info`, respectively, for $v_{1}$; and
- `v2_sc` and `v2_sc_info`, respectively, for $v_{2}$.

Set tolerances and iteration budget to their corresponding default values.

Replace the `pass` statement in the succeeding cell with your code.

In [17]:
U=(49.0/1.10)*1e3/mt.sqrt(3.0)
_V1SC0,_V1SC1=0.95*U,0.85*U
_V2SC0,_V2SC1=0.95*U,0.85*U
tol,maxit=1e-8,100
v1_sc,v1_sc_info=spo.newton(
    f,x0=_V1SC0,x1=_V1SC1,tol=tol,maxiter=maxit,full_output=True,disp=False
)
v2_sc,v2_sc_info=spo.newton(
    g,x0=_V2SC0,x1=_V2SC1,args=(v1_sc,),
    tol=tol,maxiter=maxit,full_output=True,disp=False
)

In [18]:
print("-----")
print(v1_sc_info)
print("-----")
print(v2_sc_info)

-----
      converged: True
           flag: converged
 function_calls: 9
     iterations: 8
           root: 25687.756667151712
         method: secant
-----
      converged: True
           flag: converged
 function_calls: 9
     iterations: 8
           root: 25670.61526184411
         method: secant


In [19]:
print(f"Solving for Bonnie's operating voltage using the secant method...")
print(f"\tstart: {_V1SC0/1e3:.3f}, {_V1SC1/1e3:.3f} kilovolts")
print(f"\troot: {v1_sc/1e3:.6f}")
print(f"\tresidual: {f(v1_sc):.6e}")

Solving for Bonnie's operating voltage using the secant method...
	start: 24.432, 21.861 kilovolts
	root: 25.687757
	residual: 6.400000e+01


In [20]:
print(f"Solving for Clyde's operating voltage using the secant method...")
print(f"\tstart: {_V2SC0/1e3:.3f}, {_V2SC1/1e3:.3f} kilovolts")
print(f"\troot: {v2_sc/1e3:.6f}")
print(f"\tresidual: {g(v2_sc, v1_sc):.6e}")

Solving for Clyde's operating voltage using the secant method...
	start: 24.432, 21.861 kilovolts
	root: 25.670615
	residual: -6.400000e+01


In [21]:
print(f"Phase-angle difference: {calc_angle_diff(v1_sc, v2_sc):.6f} degrees")

Phase-angle difference: -0.014429 degrees


## Instructions and reminders

### Written exam component

Write your answers in white A4 papers.
Use one (1) inch for the top, left, right, and bottom margins.
Write only on one page of a sheet.
Staple the sheets at the upper-left corner of the pages.

Corresponding maximum marks for Questions 1 - 7 are as follows.
- thirty (30) points for each of Questions 1 and 2
- ten (10) points for Question 3
- five (5) points for each of Questions 4 - 7

Question 8 is a bonus worth three (3) points.

The use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, include a brief closing section
(titled "Declaration on the use of AI tools")
declaring which and how AI tools are used in your work,
or non-use thereof.

### Programming component

Do not use any library or module other than those in the imports cell.

In addition to replacing the `pass` statements,
you also need to fix some intentional errors.

For each method,
- getting the correct value of $v_{1}$ merits ten (10) points,
- getting the correct value of $v_{2}$ merits ten (10) points,
- getting the correct value of the phase-angle difference merits ten (10) points,
  and
- getting a reasonable residual merits two (2) points.

Meeting the above conditions and all instructions
further merits a point.
Each non-compliance to an instruction, however, means a deduction of two (2) points.
Thus, one may earn up to 100 points for the programming component,
as long as
- all instructions are met,
- all code cells run properly in succession,
  and
- all computed quantities are (acceptably) equal to those in a held-out answer key.

Download this notebook file,
and save with a filename following the pattern
`EXM-01_<section>_<ID number>`,
where the section is as reflected in your Google Classroom.
For example,
if your ID number is 2013-0024
and you are enrolled in the M34W12 class,
then your notebook should be named
`EXM-01_M34W12_2013-0024.ipynb`.
Submit your notebook via the classwork platform for Exam 1 in Google Classroom.
Submissions beyond the deadline will not be considered.

The use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, please include a brief statement
(in a private comment to this classwork)
declaring which and how AI tools are used in your work,
or non-use thereof.

*Last updated by Christian Cahig on 2025-10-22*